In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer,InputSpec
import tensorflow.keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.preprocessing import image
from pathlib import Path

In [ ]:
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

In [ ]:
model_name="EfficientNetV2B2"

# Datagenarator Initialisation

In [ ]:
# train_path and val_path

train_path = '/kaggle/input/cosmys-face-datast/Comys_Hackathon5/Task_A/train'
val_path = '/kaggle/input/cosmys-face-datast/Comys_Hackathon5/Task_A/val'

# image size and batch size

image_size = 224
batch_size = 16


datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# train_dataset and val_dataset

print("\nTrain Batches:")
train_batches = datagen.flow_from_directory(
    directory=train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=True
)

print("\nTest Batches:")
val_batches = datagen.flow_from_directory(
    directory=val_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=False
)


# Model Defination

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, LSTM, Dense



efficientnet=tf.keras.applications.EfficientNetV2B2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(image_size,image_size,3),
    pooling=None,
    classes=1000
)


for layer in efficientnet.layers:
    layer.trainable=True
conv = efficientnet.output


x = GlobalAveragePooling2D()(conv)


x = Reshape((1, x.shape[-1]))(x)


x = LSTM(64, return_sequences=False)(x)


output = Dense(len(set(train_batches.classes)), activation='softmax')(x)


model = Model(inputs=efficientnet.input, outputs=output)
model.summary()




# Training Setup

In [ ]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.01, epsilon=0.1)
model.compile(optimizer=opt1,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=30, min_delta=0.001)
model_checkpoint = ModelCheckpoint(f'/kaggle/working/{model_name}.weights.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True, verbose=1)

In [ ]:
#epoch
epc=60

# Model Training

In [ ]:
history = model.fit(train_batches,
                    epochs=epc,
                  verbose=1,
                    validation_data=val_batches,callbacks=[Earlystop,model_checkpoint])

# Classification Report for Training

In [ ]:
y_true = train_batches.classes
y_pred_probs = model.predict(train_batches)
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = np.mean(y_pred == y_true)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

labels = [0, 1]

cm = confusion_matrix(y_true, y_pred, labels=labels)

print("Confusion Matrix for Training dataset:")
print(cm)
print("Accuracy for Training dataset:")
print(accuracy *100)


In [ ]:
from sklearn.metrics import classification_report
print("Training dataset:")
print(classification_report(y_true, y_pred))

# Model Testing

In [ ]:
from tensorflow.keras.preprocessing import image
from pathlib import Path
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model_1=Model(inputs=efficientnet.input, outputs=output)
model_1.load_weights(f'/kaggle/working/{model_name}.weights.h5')

In [ ]:
y_true = val_batches.classes
y_pred_probs = model_1.predict(val_batches)
y_pred = np.argmax(y_pred_probs, axis=1)
accuracy = np.mean(y_pred == y_true)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

labels = [0, 1]

cm = confusion_matrix(y_true, y_pred, labels=labels)

print("Confusion Matrix for Val Dataset:")
print(cm)
print("Accuracy for Val Dataset:")
print(accuracy *100)


In [ ]:
from sklearn.metrics import classification_report
print("Val Dataset:")
print(classification_report(y_true, y_pred))